Organizing English LibreSpeech Corpus into Manfests for Speech

In [15]:
import os
import re
import json
import glob
from pathlib import Path

from pathlib import PurePath
from pydub import AudioSegment

from nemo_text_processing.text_normalization.normalize import Normalizer
from nemo.collections import nlp as nemo_nlp



### Intitiate text normalizer and puctuator

In [16]:
normalizer = Normalizer(input_case='lower_cased', lang="en")
punctuator = nemo_nlp.models.PunctuationCapitalizationModel.from_pretrained("punctuation_en_distilbert")

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
 NeMo-text-processing :: DEBUG    :: cardinal:  0.42s -- 6247 nodes
 NeMo-text-processing :: DEBUG    :: ordinal:  1.04s -- 1478 nodes
 NeMo-text-processing :: DEBUG    :: decimal:  0.20s -- 3151 nodes
 NeMo-text-processing :: DEBUG    :: fraction:  0.19s -- 4254 nodes
 NeMo-text-processing :: DEBUG    :: measure:  9.44s -- 49430 nodes
 NeMo-text-processing :: DEBUG    :: date:  0.53s -- 4456 nodes
 NeMo-text-processing :: DEBUG    :: time:  0.12s -- 418 nodes
 NeMo-text-processing :: DEBUG    :: telephone:  0.46s -- 3467 nodes
 NeMo-text-processing :: DEBUG    :: electronic:  0.14s -- 902 nodes
 NeMo-text-processing :: DEBUG    :: money:  7.76s -- 13153 nodes
 NeMo-text-processing :: DEBUG    :: whitelist:  1.19s -- 16688 nodes
 NeMo-text-processing :: DEBUG    :: punct:  2.21s -- 259 nodes
 NeMo-text-processing :: DEBUG    :: word:  4.78s -- 1295 nodes
 NeMo-text-processing :: DEBUG    :: serial:  7.76s -- 10772 node

[NeMo I 2023-11-12 08:23:32 cloud:58] Found existing object /home/ubuntu/.cache/torch/NeMo/NeMo_1.21.0rc0/punctuation_en_distilbert/6bdea9786c4395fbbe02e4143d2e1cee/punctuation_en_distilbert.nemo.
[NeMo I 2023-11-12 08:23:32 cloud:64] Re-using file from: /home/ubuntu/.cache/torch/NeMo/NeMo_1.21.0rc0/punctuation_en_distilbert/6bdea9786c4395fbbe02e4143d2e1cee/punctuation_en_distilbert.nemo
[NeMo I 2023-11-12 08:23:32 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-11-12 08:23:34 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-uncased, vocab_file: /tmp/tmpb7mnbbwt/tokenizer.vocab_file, merges_files: None, special_tokens_dict: {}, and use_fast: False


[NeMo W 2023-11-12 08:23:35 modelPT:258] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2023-11-12 08:23:35 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_audio: false
    audio_file: null
    sample_rate: 16000
    use_bucketing: true
    batch_size: 32
    preload_audios: true
    use_tarred_dataset: false
    label_info_save_dir: null
    text_file: text_train.txt
    labels_file: labels_train.txt
    tokens_in_batch: null
    max_seq_length: 128
    num_samples: -1
    use_cache: true
    cache_dir: null
    get_label_frequences: false
    verbose: true
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    shard_strategy: scatter
    shuffle: true
    drop_last: false
    pin_memory: true
    num_workers: 8
    persistent_wor

[NeMo I 2023-11-12 08:23:38 save_restore_connector:249] Model PunctuationCapitalizationModel was successfully restored from /home/ubuntu/.cache/torch/NeMo/NeMo_1.21.0rc0/punctuation_en_distilbert/6bdea9786c4395fbbe02e4143d2e1cee/punctuation_en_distilbert.nemo.


### Start Hugging Face NLP systems

In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

entity_tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
entity_model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

hf_nlp = pipeline("ner", model=entity_model, tokenizer=entity_tokenizer, grouped_entities=True)


def tag_entities(text):

    ner_results = hf_nlp(text)
    print(ner_results)

    # example: [{'entity_group': 'PER', 'score': 0.8913538, 'word': 'Min', 'start': 0, 'end': 3}, {'entity_group': 'LOC', 'score': 0.9983326, 'word': 'West Van Buren Street', 'start': 93, 'end': 114}]
    for ner_dict in ner_results:

        entity_group = ner_dict['entity_group']
        start = ner_dict['start']
        end = ner_dict['end']
        word = ner_dict['word']

        text = text.replace(word, "B-"+entity_group+" "+word+" E-"+entity_group)

    print("ner tagged text", text)


    return text

[NeMo W 2023-11-12 08:23:50 nemo_logging:349] /home/ubuntu/anaconda3/envs/nemo/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
      warnings.warn(
    


### Start pretrained Emotion Classification system

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2FeatureExtractor
from src.models import Wav2Vec2ForSpeechClassification, HubertForSpeechClassification

emotion_model = HubertForSpeechClassification.from_pretrained("Rajaram1996/Hubert_emotion")
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
sampling_rate=16000 # defined by the model; must convert mp3 to this rate.
config = AutoConfig.from_pretrained("Rajaram1996/Hubert_emotion")

def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate, sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech


def predict(path, sampling_rate):
    speech = speech_file_to_array_fn(path, sampling_rate)
    inputs = feature_extractor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    inputs = {key: inputs[key].to(device) for key in inputs}

    with torch.no_grad():
        logits = model(**inputs).logits

    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in
               enumerate(scores)]
    return outputs

def get_emotion_labels(audio_file, sampling_rate=16000, score=50.0):
    sound_array = speech_file_to_array_fn(audio_file, sampling_rate)
    
    inputs = feature_extractor(sound_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    inputs = {key: inputs[key].to("cpu").float() for key in inputs}

    with torch.no_grad():
        logits = emotion_model(**inputs).logits

    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]

    outputs = [{
        "emo": config.id2label[i],
        "score": round(score * 100, 1)}
        for i, score in enumerate(scores)
    ]

    #[{'emo': 'female_neutral', 'score': 73.9}, {'emo': 'female_happy', 'score': 24.8}]
    emotion_labels = [row for row in sorted(outputs, key=lambda x:x["score"], reverse=True) if row['score'] != '0.0%'][:2]

    all_labels = []
    for emotion_dict in emotion_labels:
        label = emotion_dict['emo'].split("_")[1].upper()
        score = emotion_dict['score']

        if score > 50.0:
            all_labels.append(label)

    return all_labels


Some weights of the model checkpoint at Rajaram1996/Hubert_emotion were not used when initializing HubertForSpeechClassification: ['hubert.encoder.pos_conv_embed.conv.weight_v', 'hubert.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing HubertForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForSpeechClassification were not initialized from the model checkpoint at Rajaram1996/Hubert_emotion and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight

### Librespeech: Get data, un-compress it and then set paths

In [19]:
#unzip downloaded files
os.system("tar -xzvf train.tar.gz")
os.system("tar -xzvf test.tar.gz")
os.system("tar -xzvf dev.tar.gz")

tar (child): train.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
tar (child): test.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
tar (child): dev.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


512

In [20]:
#define paths to folders created afte unzipping
LIBRE = '/n/disk1/audio_datasets/EN_libre/'
TRAIN_DATA = Path(LIBRE+'/LibriSpeech/train-clean-360/')
TRAIN_DATA_WAV = str(TRAIN_DATA) + '-wav/'
os.system('mkdir -p ' + TRAIN_DATA_WAV)
TRAIN_DATA_WAV = Path(TRAIN_DATA_WAV)


DEV_DATA = Path(LIBRE+'/LibriSpeech/dev-clean/')
DEV_DATA_WAV = str(DEV_DATA) + '-wav/'
os.system('mkdir -p ' + DEV_DATA_WAV)
DEV_DATA_WAV = Path(DEV_DATA_WAV)

TEST_DATA = Path(LIBRE+'/LibriSpeech/test-clean/')
TEST_DATA_WAV = str(TEST_DATA) + '-wav/'
os.system('mkdir -p ' + TEST_DATA_WAV)
TEST_DATA_WAV = Path(TEST_DATA_WAV)

In [21]:
allpath = [TRAIN_DATA, DEV_DATA, TEST_DATA]


def normalize(text):

    text = text.lower()
    normalized = normalizer.normalize(text, verbose=True, punct_post_process=True)
    normalized = [normalized]
    norm_punctuated = punctuator.add_punctuation_capitalization(normalized)[0]
    return norm_punctuated

def read_transcription(filepath):

    trans = open(filepath,'r').readlines()
    trans_dict = {}
    for line in trans:
        line = line.strip().split()
        text = ' '.join(line[1:])
        text = normalize(text)

        trans_dict[line[0]] = text
    
    return trans_dict

def process_librispeech(datakey):

    print(datakey)
    datafolders = glob.glob(str(datakey)+'/*')
    print(datafolders)

    datakey_wav = str(datakey) + '-wav/'
    os.system('mkdir -p ' + datakey_wav)
    datakey_wav = Path(datakey_wav)

    manifest = open(datakey_wav.name+'.json','w')

    for folder in datafolders[:4]:
        sessdirs = glob.glob(folder + '/*')

        for sessdir in sessdirs:
            print("ss", sessdir)
            segments = glob.glob(sessdir + '/*')
            print("segments", segments)

            transcription = [x for x in segments if re.search(".txt", x)][0]
            trans_dict = read_transcription(transcription)
            #print(trans_dict)


            for segment in segments:
                #allfiles = glob.glob(segment + '/*')
                
                filepath = PurePath(segment)

                if ".flac" in str(filepath):
                    
                    sample_dict = {}

                    print("Old path", filepath)

                    filekey = filepath.name.replace(filepath.suffix, "")
                    transcription = trans_dict[filekey]
                    wav_filepath = str(datakey_wav) + "/" + filekey + ".wav"
                    sample_dict['audiofilepath'] = wav_filepath
                    sample_dict['text'] = transcription
                    sample_dict['tagged_text'] = transcription

                    flac_tmp_audio_data = AudioSegment.from_file(filepath, filepath.suffix[1:])
                    flac_tmp_audio_data.export(wav_filepath, format="wav")
                    sample_dict['instruction'] = "transcribe speech"

                    json.dump(sample_dict, manifest)
                    manifest.write("\n")


                    tagged_transcription = tag_entities(transcription)
                    sample_dict['text'] = transcription
                    sample_dict['tagged_text'] = tagged_transcription
                    sample_dict['instruction'] = "transcribe and mark named entities"
                    json.dump(sample_dict, manifest)
                    manifest.write("\n")


                    emotion_labels = get_emotion_labels(audio_file=wav_filepath, sampling_rate=16000)
                    print("Emotion Labels", emotion_labels)
                    emotion_labels = ' '.join(emotion_labels)

                    final_transcription = tagged_transcription + " " + emotion_labels

                    sample_dict['text'] = final_transcription
                    sample_dict['tagged_text'] = transcription
                    sample_dict['instruction'] = "transcribe, mark named entitites and track speaker emotion"
                    json.dump(sample_dict, manifest)
                    manifest.write("\n")

                    sample_dict['prompt'] = final_transcription
                    print("tagged transcription", tagged_transcription)
    
    manifest.close()

for datakey in allpath[1:2]:
    #print(datapath)
    print(datakey)
    process_librispeech(datakey)

 NeMo-text-processing :: DEBUG    :: tokens { name: "he" } tokens { name: "was" } tokens { name: "in" } tokens { name: "a" } tokens { name: "fevered" } tokens { name: "state" } tokens { name: "of" } tokens { name: "mind" } tokens { name: "owing" } tokens { name: "to" } tokens { name: "the" } tokens { name: "blight" } tokens { name: "his" } tokens { name: "wife's" } tokens { name: "action" } tokens { name: "threatened" } tokens { name: "to" } tokens { name: "cast" } tokens { name: "upon" } tokens { name: "his" } tokens { name: "entire" } tokens { name: "future" }


/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean
/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean
['/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/2277', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/2035', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/2086', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/7976', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/1988', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/777', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/84', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/1673', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/8297', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/5536', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/6345', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/6295', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/6313', '/n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/5338', '/n/disk1/

100%|██████████| 1/1 [00:00<00:00, 12.66batch/s]
 NeMo-text-processing :: DEBUG    :: tokens { name: "he" } tokens { name: "would" } tokens { name: "have" } tokens { name: "to" } tokens { name: "pay" } tokens { name: "her" } tokens { name: "the" } tokens { name: "money" } tokens { name: "which" } tokens { name: "she" } tokens { name: "would" } tokens { name: "now" } tokens { name: "regularly" } tokens { name: "demand" } tokens { name: "or" } tokens { name: "there" } tokens { name: "would" } tokens { name: "be" } tokens { name: "trouble" } tokens { name: "it" } tokens { name: "did" } tokens { name: "not" } tokens { name: "matter" } tokens { name: "what" } tokens { name: "he" } tokens { name: "did" }


[NeMo I 2023-11-12 08:24:22 punctuation_capitalization_model:1167] Using batch size 1 for inference
[NeMo I 2023-11-12 08:24:22 punctuation_capitalization_infer_dataset:127] Max length: 28
[NeMo I 2023-11-12 08:24:22 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-11-12 08:24:22 data_preprocessing:406] Min: 26 |                  Max: 26 |                  Mean: 26.0 |                  Median: 26.0
[NeMo I 2023-11-12 08:24:22 data_preprocessing:412] 75 percentile: 26.00
[NeMo I 2023-11-12 08:24:22 data_preprocessing:413] 99 percentile: 26.00


100%|██████████| 1/1 [00:00<00:00,  9.02batch/s]
 NeMo-text-processing :: DEBUG    :: tokens { name: "hurstwood" } tokens { name: "walked" } tokens { name: "the" } tokens { name: "floor" } tokens { name: "mentally" } tokens { name: "arranging" } tokens { name: "the" } tokens { name: "chief" } tokens { name: "points" } tokens { name: "of" } tokens { name: "his" } tokens { name: "situation" }


...

[NeMo I 2023-11-12 08:28:57 punctuation_capitalization_model:1167] Using batch size 1 for inference
[NeMo I 2023-11-12 08:28:57 punctuation_capitalization_infer_dataset:127] Max length: 32
[NeMo I 2023-11-12 08:28:57 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-11-12 08:28:57 data_preprocessing:406] Min: 30 |                  Max: 30 |                  Mean: 30.0 |                  Median: 30.0
[NeMo I 2023-11-12 08:28:57 data_preprocessing:412] 75 percentile: 30.00
[NeMo I 2023-11-12 08:28:57 data_preprocessing:413] 99 percentile: 30.00


100%|██████████| 1/1 [00:00<00:00, 15.26batch/s]
 NeMo-text-processing :: DEBUG    :: tokens { name: "then" } tokens { name: "they" } tokens { name: "began" } tokens { name: "to" } tokens { name: "run" } tokens { name: "and" } tokens { name: "rushing" } tokens { name: "into" } tokens { name: "the" } tokens { name: "house" } tokens { name: "they" } tokens { name: "fell" } tokens { name: "upon" } tokens { name: "their" } tokens { name: "father's" } tokens { name: "neck" }


[NeMo I 2023-11-12 08:28:57 punctuation_capitalization_model:1167] Using batch size 1 for inference
[NeMo I 2023-11-12 08:28:57 punctuation_capitalization_infer_dataset:127] Max length: 20
[NeMo I 2023-11-12 08:28:57 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-11-12 08:28:57 data_preprocessing:406] Min: 18 |                  Max: 18 |                  Mean: 18.0 |                  Median: 18.0
[NeMo I 2023-11-12 08:28:57 data_preprocessing:412] 75 percentile: 18.00
[NeMo I 2023-11-12 08:28:57 data_preprocessing:413] 99 percentile: 18.00


100%|██████████| 1/1 [00:00<00:00, 18.67batch/s]


Old path /n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/7976/110523/7976-110523-0009.flac
[]
ner tagged text We are going into the forest to hew wood, and in the evening when we are ready, we will come and fetch you again.
Emotion Labels ['HAPPY']
tagged transcription We are going into the forest to hew wood, and in the evening when we are ready, we will come and fetch you again.
Old path /n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/7976/110523/7976-110523-0007.flac
[]
ner tagged text But her husband felt heavy at heart and thought it were better to share the last crust with the children.
Emotion Labels ['NEUTRAL']
tagged transcription But her husband felt heavy at heart and thought it were better to share the last crust with the children.
Old path /n/disk1/audio_datasets/EN_libre/LibriSpeech/dev-clean/7976/110523/7976-110523-0011.flac
[]
ner tagged text Come in and stop with me, and no harm shall come to you, and so saying, she took them both by the hand and led them